In [1]:
import pandas as pd
import numpy as np
import datetime
import time
import os

### Recortes temporales en Audio
---

In [2]:
audio_path = "../Data Procesada/data_audio.csv"
audio = pd.read_csv(audio_path)
audio.head(3)

,Unnamed: 0,folder,path_audio,file,date,text,start,end
0,0,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,hola hola muy buenas tardes bienvenidos al dia...,2020-08-14 17:56:14,2020-08-14 17:56:33
1,1,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,tenemos que hablar de esta definición por part...,2020-08-14 17:56:34,2020-08-14 17:57:19
2,2,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,algún punto empieza a reconocer también y si u...,2020-08-14 17:57:19,2020-08-14 17:58:04


In [3]:
audio.shape

(317, 8)

In [4]:
audio['start'] = audio['start'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
audio['end'] = audio['end'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))


audio.dtypes

Unnamed: 0             int64
folder                object
path_audio            object
file                  object
date                  object
text                  object
start         datetime64[ns]
end           datetime64[ns]
dtype: object

In [5]:
#Seteo del intervalo delta
delta = 32
td = datetime.timedelta(minutes=delta)


In [6]:
# Join de textos en el intervalo Delta
registrosAudio =  pd.DataFrame(columns=['folder', 'file','deltaStart', 'deltaEnd','textDelta'])

for file in audio['file'].unique():
    
    maskFile =  audio['file'] == file
    
    folder = audio.loc[maskFile, 'folder'].unique()
    
    deltaStart = audio.loc[maskFile, 'start'].min()
    deltaEnd = deltaStart + td
    
    endFile = audio.loc[maskFile, 'end'].max()
    
    while deltaStart <= endFile:
        
        
        maskDelta = (audio['start'] >= deltaStart) & (audio['end'] < deltaEnd)
        
        audio.loc[maskDelta & maskFile,'deltaStart'] = deltaStart
        
        audio.loc[maskDelta & maskFile,'deltaEnd'] = deltaEnd

        text = audio.loc[maskDelta & maskFile,'text'].unique()
        textDelta = ','.join(text)
        newInput=pd.DataFrame([textDelta], columns=['textDelta'])
        newInput['words_counts'] = [len(x.split()) for x in newInput['textDelta'].tolist()]
        newInput['file'] = file
        newInput['folder'] = folder
        newInput['deltaStart'] = deltaStart
        newInput['deltaEnd'] = deltaEnd
        newInput.reset_index(inplace=True)
        
        registrosAudio = registrosAudio.append(newInput)

        deltaStart = deltaStart + td
        deltaEnd = deltaStart + td
        
#registrosAudio.drop(['index'],axis=1)
registrosAudio['file'] = registrosAudio['file'].apply(lambda x: x.split('.')[0]) 
registrosAudio.rename(columns={"folder": "fuente"}, inplace=True)
registrosAudio.head()



,fuente,file,deltaStart,deltaEnd,textDelta,index,words_counts
0,C5N,C5N2020-08-14-17-55-49,2020-08-14 17:56:14,2020-08-14 18:28:14,hola hola muy buenas tardes bienvenidos al dia...,0.0,5462.0
0,C5N,C5N2020-08-14-17-55-49,2020-08-14 18:28:14,2020-08-14 19:00:14,la gente circula porque los negocios se han ab...,0.0,3889.0
0,C5N,C5N2020-08-14-17-55-49,2020-08-14 19:00:14,2020-08-14 19:32:14,día del niño niña o totalmente inédito como ca...,0.0,4092.0
0,C5N,C5N2020-08-14-17-55-49,2020-08-14 19:32:14,2020-08-14 20:04:14,con siri poco la hipocresía y con la mentira e...,0.0,3808.0
0,TN,TN2020-08-14-18-59-25,2020-08-14 18:59:25,2020-08-14 19:31:25,de los colores que se habían ido arrojó de las...,0.0,4576.0


In [7]:
registrosAudio.shape

(8, 7)

### Recortes temporales en video
---

In [8]:
#Concatenado de las tablas de video

In [9]:
videoTablePath = "../Data/Video/TablasVideo/"
data_video = pd.DataFrame()

for videoTable in os.listdir(videoTablePath):
    registro = pd.read_csv(videoTablePath + videoTable)
    #print(registro.shape)
    data_video = pd.concat([data_video, registro])

data_video.drop(['Unnamed: 0'], axis=1, inplace=True)
data_video['file'] = data_video['file'].apply(lambda x: x if x is np.NaN else x.split('.')[0])

In [10]:
data_video.sample(3)

,personaje,fecha_hora,genero,file,fuente
4811,NO IDENTIFICADO,2020-08-14 20:06:27,M,TN2020-08-14-18-59-25,TN
2872,diego_leuco,2020-08-14 18:31:20,M,C5N2020-08-14-17-55-49,C5N
6820,NO IDENTIFICADO,2020-08-14 20:32:12,F,TN2020-08-14-18-59-25,TN


In [11]:
##Export dataset
data_video.to_csv("../Data procesada/data_video.csv")


In [12]:
#Recortes temporales

In [13]:
video_path = "../Data Procesada/data_video.csv"
video = pd.read_csv(video_path)
video.head(3)

,Unnamed: 0,personaje,fecha_hora,genero,file,fuente
0,0,SIN PERSONAS IDENTIFICABLES,2020-08-14 17:55:49,NaN,C5N2020-08-14-17-55-49,C5N
1,1,SIN PERSONAS IDENTIFICABLES,2020-08-14 17:55:50,NaN,C5N2020-08-14-17-55-49,C5N
2,2,SIN PERSONAS IDENTIFICABLES,2020-08-14 17:55:51,NaN,C5N2020-08-14-17-55-49,C5N


In [14]:
video.columns

Index(['Unnamed: 0', 'personaje', 'fecha_hora', 'genero', 'file', 'fuente'], dtype='object')

In [15]:

video['datetime'] = video['fecha_hora'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
video.drop(['Unnamed: 0', 'fecha_hora'], axis=1, inplace=True)
video.dtypes


personaje            object
genero               object
file                 object
fuente               object
datetime     datetime64[ns]
dtype: object

In [16]:
video.head(2)

,personaje,genero,file,fuente,datetime
0,SIN PERSONAS IDENTIFICABLES,NaN,C5N2020-08-14-17-55-49,C5N,2020-08-14 17:55:49
1,SIN PERSONAS IDENTIFICABLES,NaN,C5N2020-08-14-17-55-49,C5N,2020-08-14 17:55:50


In [17]:

registros =  pd.DataFrame()

for file in video['file'].unique():
    
    maskFile =  video['file'] == file
    
    deltaStart = video.loc[maskFile, 'datetime'].min()
    deltaEnd = deltaStart + td
    
    endFile = video.loc[maskFile, 'datetime'].max()
    
    while deltaStart <= endFile:
        
        
        maskDelta = (video['datetime'] >= deltaStart) & (video['datetime'] < deltaEnd)
        
        video.loc[maskDelta & maskFile,'deltaStart'] = deltaStart
        
        video.loc[maskDelta & maskFile,'deltaEnd'] = deltaEnd

        newInput = video.loc[maskDelta & maskFile, ['deltaStart', 'deltaEnd', 'file','fuente','personaje']].value_counts()
        
        newInput = pd.DataFrame(newInput)
        newInput.reset_index(inplace=True)
        
        registros = registros.append(newInput)

        deltaStart = deltaStart + td
        deltaEnd = deltaStart + td
        
registros


,deltaStart,deltaEnd,file,fuente,personaje,0
0,2020-08-14 17:55:49,2020-08-14 18:27:49,C5N2020-08-14-17-55-49,C5N,NO IDENTIFICADO,1855
1,2020-08-14 17:55:49,2020-08-14 18:27:49,C5N2020-08-14-17-55-49,C5N,SIN PERSONAS IDENTIFICABLES,360
2,2020-08-14 17:55:49,2020-08-14 18:27:49,C5N2020-08-14-17-55-49,C5N,alberto_fernandez,149
3,2020-08-14 17:55:49,2020-08-14 18:27:49,C5N2020-08-14-17-55-49,C5N,horacio_rodriguez_larreta,130
4,2020-08-14 17:55:49,2020-08-14 18:27:49,C5N2020-08-14-17-55-49,C5N,franco_mercuriali,31
...,...,...,...,...,...,...
10,2020-08-14 20:35:25,2020-08-14 21:07:25,TN2020-08-14-18-59-25,TN,carolina_piparo,2
11,2020-08-14 20:35:25,2020-08-14 21:07:25,TN2020-08-14-18-59-25,TN,liilta_carrio,1
12,2020-08-14 20:35:25,2020-08-14 21:07:25,TN2020-08-14-18-59-25,TN,guillermo_lobo,1
13,2020-08-14 20:35:25,2020-08-14 21:07:25,TN2020-08-14-18-59-25,TN,alberto_fernandez,1


In [18]:
video['datetime'].min()

Timestamp('2020-08-14 17:55:49')

In [19]:
registrosVideo = registros.pivot(index=['deltaStart', 'deltaEnd', 'fuente', 'file'], columns=['personaje'])
#registrosVideo.reset_index(inplace=True)
registrosVideo

0  \
personaje                                                              lionel_messi   
deltaStart          deltaEnd            fuente file                                   
2020-08-14 17:55:49 2020-08-14 18:27:49 C5N    C5N2020-08-14-17-55-49           NaN   
2020-08-14 18:27:49 2020-08-14 18:59:49 C5N    C5N2020-08-14-17-55-49           NaN   
2020-08-14 18:59:25 2020-08-14 19:31:25 TN     TN2020-08-14-18-59-25            NaN   
2020-08-14 18:59:49 2020-08-14 19:31:49 C5N    C5N2020-08-14-17-55-49           NaN   
2020-08-14 19:31:25 2020-08-14 20:03:25 TN     TN2020-08-14-18-59-25            NaN   
2020-08-14 19:31:49 2020-08-14 20:03:49 C5N    C5N2020-08-14-17-55-49           1.0   
2020-08-14 20:03:25 2020-08-14 20:35:25 TN     TN2020-08-14-18-59-25            NaN   
2020-08-14 20:35:25 2020-08-14 21:07:25 TN     TN2020-08-14-18-59-25            1.0   

                                                                                       \
personaje                                                             NO IDENTIFICADO   
deltaStart          deltaEnd            fuente file                                     
2020-08-14 17:55:49 2020-08-14 18:27:49 C5N    C5N2020-08-14-17-55-49          1855.0   
2020-08-14 18:27:49 2020-08-14 18:59:49 C5N    C5N2020-08-14-17-55-49          1556.0   
2020-08-14 18:59:25 2020-08-14 19:31:25 TN     TN2020-08-14-18-59-25            679.0   
2020-08-14 18:59:49 2020-08-14 19:31:49 C5N    C5N2020-08-14-17-55-49           899.0   
2020-08-14 19:31:25 2020-08-14 20:03:25 TN     TN2020-08-14-18-59-25            387.0   
2020-08-14 19:31:49 2020-08-14 20:03:49 C5N    C5N2020-08-14-17-55-49          1046.0   
2020-08-14 20:03:25 2020-08-14 20:35:25 TN     TN2020-08-14-18-59-25            857.0   
2020-08-14 20:35:25 2020-08-14 21:07:25 TN     TN2020-08-14-18-59-25            756.0   

                                                                                                   \
personaje                                                             SIN PERSONAS IDENTIFICABLES   
deltaStart          deltaEnd            fuente file                                                 
2020-08-14 17:55:49 2020-08-14 18:27:49 C5N    C5N2020-08-14-17-55-49                       360.0   
2020-08-14 18:27:49 2020-08-14 18:59:49 C5N    C5N2020-08-14-17-55-49                       686.0   
2020-08-14 18:59:25 2020-08-14 19:31:25 TN     TN2020-08-14-18-59-25                        732.0   
2020-08-14 18:59:49 2020-08-14 19:31:49 C5N    C5N2020-08-14-17-55-49                       895.0   
2020-08-14 19:31:25 2020-08-14 20:03:25 TN     TN2020-08-14-18-59-25                        677.0   
2020-08-14 19:31:49 2020-08-14 20:03:49 C5N    C5N2020-08-14-17-55-49                       509.0   
2020-08-14 20:03:25 2020-08-14 20:35:25 TN     TN2020-08-14-18-59-25                        399.0   
2020-08-14 20:35:25 2020-08-14 21:07:25 TN     TN2020-08-14-18-59-25                        628.0   

                                                                                        \
personaje                                                             abigail_lassalle   
deltaStart          deltaEnd            fuente file                                      
2020-08-14 17:55:49 2020-08-14 18:27:49 C5N    C5N2020-08-14-17-55-49              NaN   
2020-08-14 18:27:49 2020-08-14 18:59:49 C5N    C5N2020-08-14-17-55-49              NaN   
2020-08-14 18:59:25 2020-08-14 19:31:25 TN     TN2020-08-14-18-59-25               1.0   
2020-08-14 18:59:49 2020-08-14 19:31:49 C5N    C5N2020-08-14-17-55-49              NaN   
2020-08-14 19:31:25 2020-08-14 20:03:25 TN     TN2020-08-14-18-59-25               NaN   
2020-08-14 19:31:49 2020-08-14 20:03:49 C5N    C5N2020-08-14-17-55-49              NaN   
2020-08-14 20:03:25 2020-08-14 20:35:25 TN     TN2020-08-14-18-59-25               NaN   
2020-08-14 20:35:25 2020-08-14 21:07:25 TN     TN2020-08-14-18-59-25               NaN   

                                         

In [20]:
registrosVideo = registrosVideo.droplevel(0, axis=1)
registrosVideo

,,,personaje,lionel_messi,NO IDENTIFICADO,SIN PERSONAS IDENTIFICABLES,abigail_lassalle,alberto_fernandez,alexis_szewczyk,carolina_amoroso,carolina_piparo,christian_castillo,cristina_kirchner,...,mauricio_macri,nicolas_winazki,nicolas_winazki_1,ricardo_canaletti,rodrigo_alvarez,rolando_barbano,santiago_fioriti,santiago_teran,sergio_berensztein,soledad_acuna
deltaStart,deltaEnd,fuente,file,,,,,,,,,,,,,,,,,,,,,
2020-08-14 17:55:49,2020-08-14 18:27:49,C5N,C5N2020-08-14-17-55-49,NaN,1855.0,360.0,NaN,149.0,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,2.0,NaN,8.0,1.0,NaN,NaN,1.0
2020-08-14 18:27:49,2020-08-14 18:59:49,C5N,C5N2020-08-14-17-55-49,NaN,1556.0,686.0,NaN,127.0,NaN,NaN,NaN,1.0,NaN,...,4.0,NaN,2.0,1.0,NaN,3.0,1.0,1.0,NaN,1.0
2020-08-14 18:59:25,2020-08-14 19:31:25,TN,TN2020-08-14-18-59-25,NaN,679.0,732.0,1.0,24.0,NaN,125.0,4.0,2.0,3.0,...,NaN,70.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,3.0
2020-08-14 18:59:49,2020-08-14 19:31:49,C5N,C5N2020-08-14-17-55-49,NaN,899.0,895.0,NaN,68.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,3.0,3.0,NaN,1.0,1.0,NaN
2020-08-14 19:31:25,2020-08-14 20:03:25,TN,TN2020-08-14-18-59-25,NaN,387.0,677.0,NaN,31.0,1.0,362.0,1.0,1.0,1.0,...,1.0,303.0,NaN,NaN,NaN,1.0,NaN,NaN,5.0,NaN
2020-08-14 19:31:49,2020-08-14 20:03:49,C5N,C5N2020-08-14-17-55-49,1.0,1046.0,509.0,NaN,66.0,3.0,1.0,NaN,1.0,NaN,...,18.0,2.0,1.0,NaN,1.0,6.0,31.0,NaN,2.0,NaN
2020-08-14 20:03:25,2020-08-14 20:35:25,TN,TN2020-08-14-18-59-25,NaN,857.0,399.0,NaN,122.0,NaN,92.0,1.0,2.0,NaN,...,NaN,678.0,4.0,1.0,NaN,1.0,NaN,NaN,219.0,1.0
2020-08-14 20:35:25,2020-08-14 21:07:25,TN,TN2020-08-14-18-59-25,1.0,756.0,628.0,NaN,1.0,NaN,91.0,2.0,NaN,NaN,...,NaN,239.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
dataFinal = pd.merge(registrosAudio, registrosVideo, how='left', on=['file','deltaStart', 'deltaEnd'])
dataFinal.sample()

,fuente,file,deltaStart,deltaEnd,textDelta,index,words_counts,lionel_messi,NO IDENTIFICADO,SIN PERSONAS IDENTIFICABLES,...,mauricio_macri,nicolas_winazki,nicolas_winazki_1,ricardo_canaletti,rodrigo_alvarez,rolando_barbano,santiago_fioriti,santiago_teran,sergio_berensztein,soledad_acuna
2,C5N,C5N2020-08-14-17-55-49,2020-08-14 19:00:14,2020-08-14 19:32:14,día del niño niña o totalmente inédito como ca...,0.0,4092.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
dataFinal.reset_index(inplace=True)

In [23]:
dataFinal.drop(['level_0', 'index'], axis=1, inplace=True)
dataFinal

,fuente,file,deltaStart,deltaEnd,textDelta,words_counts,lionel_messi,NO IDENTIFICADO,SIN PERSONAS IDENTIFICABLES,abigail_lassalle,...,mauricio_macri,nicolas_winazki,nicolas_winazki_1,ricardo_canaletti,rodrigo_alvarez,rolando_barbano,santiago_fioriti,santiago_teran,sergio_berensztein,soledad_acuna
0,C5N,C5N2020-08-14-17-55-49,2020-08-14 17:56:14,2020-08-14 18:28:14,hola hola muy buenas tardes bienvenidos al dia...,5462.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C5N,C5N2020-08-14-17-55-49,2020-08-14 18:28:14,2020-08-14 19:00:14,la gente circula porque los negocios se han ab...,3889.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C5N,C5N2020-08-14-17-55-49,2020-08-14 19:00:14,2020-08-14 19:32:14,día del niño niña o totalmente inédito como ca...,4092.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C5N,C5N2020-08-14-17-55-49,2020-08-14 19:32:14,2020-08-14 20:04:14,con siri poco la hipocresía y con la mentira e...,3808.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TN,TN2020-08-14-18-59-25,2020-08-14 18:59:25,2020-08-14 19:31:25,de los colores que se habían ido arrojó de las...,4576.0,NaN,679.0,732.0,1.0,...,NaN,70.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,3.0
5,TN,TN2020-08-14-18-59-25,2020-08-14 19:31:25,2020-08-14 20:03:25,pregunta relacionada con esta es posible eso s...,4632.0,NaN,387.0,677.0,NaN,...,1.0,303.0,NaN,NaN,NaN,1.0,NaN,NaN,5.0,NaN
6,TN,TN2020-08-14-18-59-25,2020-08-14 20:03:25,2020-08-14 20:35:25,digamos es una oportunidad desperdiciada básic...,4047.0,NaN,857.0,399.0,NaN,...,NaN,678.0,4.0,1.0,NaN,1.0,NaN,NaN,219.0,1.0
7,TN,TN2020-08-14-18-59-25,2020-08-14 20:35:25,2020-08-14 21:07:25,nosotros sabemos que cerca de cinco mil chicos...,3078.0,1.0,756.0,628.0,NaN,...,NaN,239.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
##Export datasetFinal
dataFinal.to_csv("../Data procesada/data_final.csv")



In [25]:
dataFinal['words_counts'].mean()

4198.0